In [0]:
%python
# ============================================================
# 1. Imports MLflow
# ============================================================

import mlflow
import mlflow.sklearn

# ============================================================
# 2. Chemin du modèle MLflow (adapter si différent)
# ============================================================

model_uri = "models:/SpaceshipRF/1"   # version 1 du modèle enregistré
# Si tu veux toujours la dernière version :
# model_uri = "models:/SpaceshipRF/Production"

# ============================================================
# 3. Charger le modèle depuis MLflow
# ============================================================

loaded_model = mlflow.sklearn.load_model(model_uri)
print("Modèle MLflow chargé avec succès !")

# ============================================================
# 4. Charger les données de test (Spark → Pandas)
# ============================================================

test_df_spark = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/ngow_lakehouse/ml_sandbox/data/test.csv")

# IMPORTANT : même nettoyage que train !
cols_to_remove = ["PassengerId", "Cabin", "Name"]
for col in cols_to_remove:
    if col in test_df_spark.columns:
        test_df_spark = test_df_spark.drop(col)

test_pd = test_df_spark.toPandas()

# ============================================================
# 5. Faire les prédictions avec le modèle MLflow
# ============================================================

predictions = loaded_model.predict(test_pd)

# ============================================================
# 6. Ajouter les prédictions au DataFrame de test
# ============================================================

test_pd["Predicted_HomePlanet"] = predictions

# Afficher les premières lignes
test_pd.head()

# ============================================================
# 7. (OPTIONNEL) Sauvegarder les prédictions dans un CSV
# ============================================================

output_path = "/Volumes/ngow_lakehouse/ml_sandbox/data/output/predictions.csv"
dbutils.fs.mkdirs("/Volumes/ngow_lakehouse/ml_sandbox/data/output")
test_pd.to_csv(output_path, index=False)

print("Prédictions sauvegardées dans :", output_path)